# SEM

## See some of these (Statistics of DOOM)
https://statisticsofdoom.com/page/structural-equation-modeling/

## Use Lavaan
https://www.lavaan.ugent.be/

## And Semplot
http://sachaepskamp.com/semPlot

## Another tutorial
https://stats.idre.ucla.edu/r/seminars/rsem/

In [1]:
library(tidyverse)
library(lavaan)
library(lubridate)
library(semPlot)

#ready_path <- '/Users/gmaurer/GD_gmaurer@nmsu/_current/gc_ltreb_io/data/'
ready_path <- '../data_tmp/'
#out_path <- '/Users/gmaurer/GD_gmaurer@nmsu/_current/gc_ltreb_io/'
out_path <- '../out_tmp/'

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(lavaan): there is no package called ‘lavaan’


In [2]:
# Read in the production data
# This comes from package knb-lter-jrn.210349001 on EDI
df <- read_csv(paste0(ready_path, 'jrn349001_cover_and_biomass_juntar.csv'),
               col_types = list(ppt_trt = col_character())) %>%
    mutate(PPT_treatment = case_match(ppt_trt,
                                      '0.2' ~ '-80%', 
                                      '1.0' ~ 'ambient',
                                      '1.8' ~ '+80%'))
str(df)

ERROR: Error: object 'ready_path' not found


In [ ]:
# Load precipitation data
# This comes from package knb-lter-jrn.210349002 on EDI
ppt <- read_csv(paste0(ready_path, 'seasonal_ppt_juntar.csv'),
                col_types = list(ppt_trt = col_character())) %>%
    mutate(PPT_treatment = case_match(ppt_trt,
                                      '0.2' ~ '-80%', 
                                      '1.0' ~ 'ambient',
                                      '1.8' ~ '+80%'))
# Split the seasons
gs_ppt <- subset(ppt, season=='growing_season')
ann_ppt <- subset(ppt, season=='annual')
# Rename columns so they can be merged
gs_ppt <- gs_ppt %>% rename("gs_ppt_mm" = "ppt_mm")
ann_ppt <- ann_ppt %>% rename("ann_ppt_mm" = "ppt_mm")
str(gs_ppt)

year,plotid,ppt_trt,n_trt,biomass.grass,biomass.rare,biomass.shrub,cover.grass,cover.rare,cover.shrub,biomass.total,cover.total,PPT treatment,ann_PPT_mm,gs_PPT_mm,shrubratio,RUEann_tot,RUEgs_tot,RUEgs_shrub,RUEgs_grass
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2006,104,1,1,126.83714,3.382477,19.691052,0.4794267,0.01832229,0.10666298,149.9107,0.6044120,-80%,NA,NA,0.13135191,NA,NA,NA,NA
2006,108,1,0,104.61701,21.647286,23.448126,0.3954378,0.11725955,0.12701439,149.7124,0.6397117,-80%,NA,NA,0.15662111,NA,NA,NA,NA
2006,114,1,1,87.40713,24.078834,17.116412,0.3303868,0.13043082,0.09271660,128.6024,0.5535342,-80%,NA,NA,0.13309561,NA,NA,NA,NA
2006,126,1,0,122.05797,36.484190,51.738546,0.4613621,0.19762846,0.28025863,210.2807,0.9392492,-80%,NA,NA,0.24604514,NA,NA,NA,NA
2006,134,1,0,121.31113,18.787405,6.804681,0.4585392,0.10176808,0.03685977,146.9032,0.5971670,-80%,NA,NA,0.04632085,NA,NA,NA,NA
2006,136,1,1,140.67847,21.607569,13.863992,0.5317451,0.11704441,0.07509881,176.1500,0.7238883,-80%,NA,NA,0.07870559,NA,NA,NA,NA


In [ ]:
# Merge precip data with plot data and calculate shrub ratio and RUE
df <- merge(df, ann_ppt[,c('year','PPT_treatment','ann_ppt_mm')], on=c('year', 'PPT_treatment'), all.x=T)
df <- merge(df, gs_ppt[,c('year','PPT_treatment','gs_ppt_mm')], on=c('year', 'PPT_treatment'), all.x=T)
str(df)

Rows: 1967760 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (3): variable, depth, Treatment
dbl  (6): value, plotid, n_trt, ppt_trt, month, year
dttm (1): datetime

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Mediation 2, growing season ppt

In [ ]:
med.model.2gs <- "
  biomass.grass ~ gs_PPT_mm  
  biomass.shrub ~ gs_PPT_mm + biomass.grass
"
med.2gs.fit <- sem(med.model.2gs, data=df)

In [ ]:
summary(med.2gs.fit, rsq=TRUE, standardized=F, fit.measures=F)

lhs,op,rhs,exo,est,se,z,pvalue
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
biomass.grass,~,gs_PPT_mm,0,2.351301e-01,0.01435420,16.3805759,0.000000
biomass.shrub,~,gs_PPT_mm,0,-1.325717e-03,0.01187173,-0.1116701,0.911085
biomass.shrub,~,biomass.grass,0,-4.455797e-01,0.02645550,-16.8426100,0.000000
biomass.grass,~~,biomass.grass,0,2.307273e+03,122.54366277,18.8281704,0.000000
biomass.shrub,~~,biomass.shrub,0,1.144925e+03,60.80916634,18.8281704,0.000000
gs_PPT_mm,~~,gs_PPT_mm,1,1.579409e+04,0.00000000,NA,NA
biomass.grass,r2,biomass.grass,0,2.745491e-01,NA,NA,NA
biomass.shrub,r2,biomass.shrub,0,3.570705e-01,NA,NA,NA


In [ ]:
#png(paste0(out_path, 'fig2sem_med2_gs_2006-2024.png'), width=4, height=4, units='in', res=400)
#pdf(paste0(out_path, 'fig2sem_med2_gs_2006-2024.pdf'), width=4, height=4)
med2gs_plot <- semPlot::semPaths(med.2gs.fit, "par",
             sizeMan = 15, sizeInt = 15, sizeLat = 15,
             edge.label.cex=1.5,
             fade=FALSE, residuals=FALSE,
             nodeLabels=c('Grass NPP', 'Shrub NPP','PPT_gs'))
#dev.off()

pdf 
  2